In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv('rawdata.xlsx - inputsheet.csv')

In [3]:
raw_data.head()

,date,time,sensor,location,number,activity,position,location.1
0,1/16/2024,10:10:30 AM,0.0,A1,1,placed,inside,A1
1,1/16/2024,10:12:30 AM,1.0,A2,1,picked,inside,A2
2,1/16/2024,10:28:30 AM,1.0,A3,1,picked,Inside,A3
3,1/16/2024,10:38:20 AM,0.0,A4,1,placed,Inside,A4
4,1/16/2024,10:45:20 AM,1.0,A5,1,placed,inside,A5


In [4]:
raw_data['datetime'] = raw_data['date'] + ' ' + raw_data['time'].str[:-3]
raw_data['datetime'] = pd.to_datetime(raw_data['datetime'], format='%m/%d/%Y %H:%M:%S')

In [5]:
raw_data.sort_values(by=['location', 'datetime'], inplace=True)
raw_data['duration'] = raw_data.groupby('location')['datetime'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

In [6]:
inside_duration = raw_data[raw_data['position'] == 'inside'].groupby('date')['duration'].sum().reset_index(name='inside_duration')
outside_duration = raw_data[raw_data['position'] == 'outside'].groupby('date')['duration'].sum().reset_index(name='outside_duration')

In [7]:
pick_activities = raw_data[raw_data['activity'] == 'picked'].groupby('date').size().reset_index(name='pick_activities')
place_activities = raw_data[raw_data['activity'] == 'placed'].groupby('date').size().reset_index(name='place_activities')

In [8]:
result = pd.merge(inside_duration, outside_duration, on='date', how='outer')
result = pd.merge(result, pick_activities, on='date', how='outer')
result = pd.merge(result, place_activities, on='date', how='outer')

In [9]:
result.fillna(0, inplace=True)

In [10]:
result.to_csv('output.csv', index=False)